<a href="https://colab.research.google.com/github/sean-otoole/HODD/blob/main/PDTx_pipeline/PDTx_03_FineTuning_DA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
#fixes a compatibility issue with the PrecollatorForGeneAndCellClassification class

!pip install --upgrade transformers==4.41
pip install peft==0.10.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 100.5 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.40.0
    Uninstalling transformers-4.40.0:
      Successfully uninstalled transformers-4.40.0


In [1]:
# first mount the drive

from google.colab import drive
import os

# Mount Google Drive to access files
drive.mount('/content/drive')

# Change the working directory to the project folder in Google Drive
os.chdir("/content/drive/MyDrive/HODD/")

# Install Git Large File Storage (LFS) for handling large files in Git repositories
# !git lfs install

#Clone the Geneformer repository (commented out to avoid repeated cloning)
# !git clone https://huggingface.co/ctheodoris/Geneformer

# Navigate to the Geneformer directory
%cd Geneformer

# Install Geneformer package locally
# I found that installing Genformer first helped with a lot of the version conflict issues
!pip install .

# Install required libraries without outputting installation logs
!pip install anndata scanpy tdigest datasets

# Import necessary modules and libraries
import numpy
import transformers
import sklearn
import pickle
import sklearn

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/HODD/Geneformer
Processing /content/drive/MyDrive/HODD/Geneformer
  Preparing metadata (setup.py) ... done
  Created wheel for geneformer: filename=geneformer-0.1.0-py3-none-any.whl size=2107997 sha256=fa4254bdde9e940e596c916e452c6020ddbe180161a84b6a9bb89fdfa33312a3
  Stored in directory: /tmp/pip-ephem-wheel-cache-stldh_fg/wheels/cb/4a/82/85f3ca798e83d69ac023bfad48225e88e8fb2b2d2a968a51b0
Successfully built geneformer
  Attempting uninstall: geneformer
    Found existing installation: geneformer 0.1.0
    Uninstalling geneformer-0.1.0:
      Successfully uninstalled geneformer-0.1.0


In [8]:
#datsets imports for handling tokenized dataset
from datasets import load_dataset, load_from_disk, ClassLabel

#load the training data
park_data_path = '/content/drive/MyDrive/HODD/Geneformer_parkinsons_files/dopamine_subset_tokenized.dataset/'
park_data = load_from_disk(park_data_path)

# Get the unique values from the "disease_state_numeric" column
classes = list(set(park_data["disease_state_numeric"]))

# make the disease state column a classlabel
park_data = park_data.cast_column("disease_state_numeric", ClassLabel(names=classes))  #convert the disease state numeric column to a class label

split_data = park_data.train_test_split(test_size=0.2, seed=42, stratify_by_column="disease_state_numeric") #establish test and train with stratification
split_data = split_data.rename_column("disease_state_numeric", "label")  # for compatibility with classifier_utils call in classifer.train_classifier

# split test and train
train_data = split_data['train']
test_data = split_data['test']

# split train into training and evaluation set
train_data, eval_data = split_data['train'].train_test_split(test_size=0.2, seed=42, stratify_by_column="label").values()

print(len(train_data))
print(len(eval_data))
print(len(test_data))

4137
1035
1294


In [9]:
from geneformer import Classifier

# training_args takes a dictionary, will use for improving the training

classifier = Classifier(classifier="cell",
                        cell_state_dict = {"state_key": "disease_state_numeric", "states": "all"},
                        forward_batch_size=200,
                        nproc=1)

output_directory = '/content/drive/MyDrive/HODD/models/'
model_directory = '/content/drive/MyDrive/HODD/Geneformer/gf-12L-95M-i4096'

os.environ["WANDB_DISABLED"] = "true"

classifier.train_classifier(
        model_directory = model_directory,
        num_classes = 2,
        train_data = train_data,
        eval_data = eval_data,
        output_directory = output_directory,
        predict=True,
    )

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/HODD/Geneformer/gf-12L-95M-i4096 and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/content/drive/MyDrive/HODD/Geneformer/geneformer/collator_for_classification.py:644: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or

Epoch,Training Loss,Validation Loss,Accuracy,Macro F1
1,0.105600,0.166633,0.962319,0.955702
2,0.116700,0.095604,0.979710,0.976737
3,0.024800,0.182665,0.969082,0.963748
4,0.022200,0.092981,0.983575,0.981016
5,0.000100,0.107839,0.984541,0.982262
6,0.006600,0.132989,0.981643,0.978817
7,0.000000,0.132494,0.982609,0.979948
8,0.000100,0.125272,0.982609,0.979849
9,0.000000,0.132502,0.982609,0.979981
10,0.000000,0.137009,0.982609,0.979981


/content/drive/MyDrive/HODD/Geneformer/geneformer/collator_for_classification.py:644: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch = {k: torch.tensor(v, dtype=torch.int64) for k, v in batch.items()}
/content/drive/MyDrive/HODD/Geneformer/geneformer/collator_for_classification.py:644: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch = {k: torch.tensor(v, dtype=torch.int64) for k, v in batch.items()}
/content/drive/MyDrive/HODD/Geneformer/geneformer/collator_for_classification.py:644: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  

In [36]:
#prepare data has an issue with the label column being present
# test_data = test_data.rename_column("label", "disease_state_numeric")

output_dir = output_directory
output_prefix = 'test'
test_data.save_to_disk(f"{output_dir}'test_data.dataset")
# test_data.save_to_disk(f"{output_dir}/{output_prefix}_labeled_test.dataset")

# Save the test data to a distinct file name
# test_data.save_to_disk(f"{output_dir}{output_prefix}_original_labeled_test.dataset")

classifier.prepare_data(input_data_file=f"{output_dir}'test_data.dataset",
                output_directory=output_dir,
                output_prefix=output_prefix)

all_metrics_test = classifier.evaluate_saved_model(
        model_directory=output_directory,
        id_class_dict_file=f"{output_dir}{output_prefix}_id_class_dict.pkl",
        test_data_file=f"{output_dir}{output_prefix}_labeled_test.dataset",
        output_directory=output_dir,
        output_prefix=output_prefix,
    )

  0%|          | 0/1 [00:00<?, ?it/s]

In [37]:
classifier.plot_conf_mat(
        conf_mat_dict={"Geneformer": all_metrics_test["conf_matrix"]},
        output_directory=output_dir,
        output_prefix=output_prefix,
        custom_class_order=["nf","hcm","dcm"],
)

ValueError: Input contains NaN.

<Figure size 1000x1000 with 0 Axes>

In [38]:
all_metrics_test["conf_matrix"]

,0,1
0,95,1
1,1,33


from matplotlib import pyplot as plt
_df_0[0].plot(kind='hist', bins=20, title=0)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1[1].plot(kind='hist', bins=20, title=1)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2.plot(kind='scatter', x=0, y=1, s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series[1]
  ys = series[0]
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_3.sort_values(1, ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel(1)
_ = plt.ylabel(0)

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series[1]
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 1}, axis=1)
              .sort_values(1, ascending=True))
  xs = counted[1]
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_4.sort_values(1, ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel(1)
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
_df_5[0].plot(kind='line', figsize=(8, 4), title=0)
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_6[1].plot(kind='line', figsize=(8, 4), title=1)
plt.gca().spines[['top', 'right']].set_visible(False)

In [ ]:
# now we evaluate the model with the test set

evaluate_saved_model(
        model_directory = ,
        id_class_dict_file,
        test_data_file,
        output_directory,
        output_prefix,
        predict=True,
    )

In [5]:
# now we plot the confusion matrix

plot_conf_mat(
        self,
        conf_mat_dict,
        output_directory,
        output_prefix,
        custom_class_order=None,
    )

'4.41.0'

In [6]:
# plot the ROC as well

plot_roc(
        self,
        roc_metric_dict,
        model_style_dict,
        title,
        output_directory,
        output_prefix,
    )

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 16.2 MB/s eta 0:00:00
  Attempting uninstall: peft
    Found existing installation: peft 0.14.0
    Uninstalling peft-0.14.0:
      Successfully uninstalled peft-0.14.0


In [ ]:
#

plot_predictions(
        predictions_file,
        id_class_dict_file,
        title,
        output_directory,
        output_prefix,
        custom_class_order=None,
        kwargs_dict=None,
    )

In [ ]:
# select the top highly expressed genes (top 1000?)

In [ ]:
# in silico perturbation

In [ ]:
# extract embeddings function and calculate cosine similarity on the embeddings healthy vs diseased

In [ ]:
# plot the distribution of the cosine similarities - should be similar to biorxiv papers critical of LLMs for research..